In [1]:
import os
import logging
import json
import re
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain_experimental.tools import PythonREPLTool
from langchain.hub import pull as hub_pull
from PyPDF2 import PdfReader  

In [2]:
# Carrega variáveis de ambiente
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Verifica a chave da API
if not api_key:
    raise ValueError("OPENAI_API_KEY não encontrada nas variáveis de ambiente. Verifique seu arquivo .env.")

# Configuração de logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# Inicializa o LLM com configurações especificadas
def initialize_llm():
    return ChatOpenAI(
        temperature=0.1,
        model="gpt-4o-mini",
        api_key=api_key,
        verbose=True
    )

# Cria o agente ReAct com suporte para execução em Python
def create_react_agent(llm):
    try:
        react_prompt_template = hub_pull("hwchase17/react")
        tools = [PythonREPLTool()]
        agent = initialize_agent(
            tools,
            llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True,
            agent_kwargs={"prompt": react_prompt_template}
        )
        return agent
    except Exception as e:
        logger.error("Erro ao criar o agente ReAct: %s", e)
        raise

# Função para processar texto e extrair informações em JSON
def process_text(texto):
    question = f"""
**IDENTIDADE**:
Você é um Tech Recruiter especializado em extração de informações estruturadas e análise precisa de currículos.

**OBJETIVO**:
Extrair informações completas e detalhadas do texto do CV e organizá-las em um formato JSON estruturado e claro.
Responder em Português Brasil

**TEXTO DO CV**:
{texto}

**INSTRUÇÕES**:
- Extraia **todas as informações relevantes** do CV.
- Retorne o resultado como um JSON bem estruturado, conforme o modelo fornecido.
- Use descrições diretas e concisas.
- Se algum dado estiver ausente, preencha com uma string vazia ("") ou uma lista vazia ([]), conforme aplicável.
- Assegure-se de que o JSON seja legível e bem formatado.

**FORMATO DE RESPOSTA**:
- Utilize **PythonREPLTool** para gerar uma lista de dicionários Python seguindo o exemplo de estrutura abaixo:
{{
    "informacoes_pessoais": {{
        "nome": "Nome Completo",
        "cidade": "Cidade, Estado/País",
        "bairro": "Bairro Opcional",
        "email": "email@exemplo.com",
        "telefone": "Telefone Opcional",
        "cargo": "Cargo Atual ou Desejado"
    }},
    "resumo_qualificacoes": [{{
        "resumo": "Visão geral profissional breve",
        "qualificacoes_chave": [
            {{"qualificacao": "Habilidade ou realização importante"}},
            {{"qualificacao": "Outra habilidade importante"}}
        ]
    }}],
    "experiencia_profissional": [
        {{
            "empresa": "Nome da Empresa",
            "cargo": "Título do Cargo",
            "periodo": "Data de Início - Data de Término",
            "atividades": [
                {{"atividade": "Responsabilidade ou realização chave"}},
                {{"atividade": "Outra responsabilidade importante"}}
            ],
            "projetos": [
                {{"titulo": "Nome do Projeto", "descricao": "Descrição do projeto"}}
            ]
        }}
    ],
    "educacao": [
        {{
            "instituicao": "Nome da Escola/Universidade",
            "grau": "Grau ou Certificação",
            "ano_formatura": "Ano"
        }}
    ],
    "certificacoes": [
        {{"certificado": "Nome da Certificação"}}
    ]
}}

**REQUISITOS FINAIS**:
- Retorne apenas o JSON sem aspas adicionais, sem palavras ou caracteres extras como "'''python" ou "'''json", e sem variáveis ou explicações.
- Certifique-se de que o JSON gerado esteja em formato de lista de dicionários, diretamente utilizável.
"""
    try:
        llm = initialize_llm()
        agent = create_react_agent(llm)
        response = agent.run(question)
        
        # Parse JSON response string
        dados_curriculo = json.loads(response)
        return dados_curriculo
        
    except Exception as e:
        logger.error("Erro ao processar o texto: %s", e)
        return {"error": str(e)}

# Função para limpar e normalizar o texto extraído
def limpar_texto(texto):
    texto = re.sub(r'\s+', ' ', texto)
    texto = re.sub(r'\n*Página \d+ de \d+\n*', '', texto)
    texto = re.sub(r'\n{3,}', '\n\n', texto)
    return texto.strip()

# Função para extrair texto de um arquivo PDF
def extrair_texto_de_pdf(caminho_pdf):
    try:
        leitor_pdf = PdfReader(caminho_pdf)
        texto = "\n".join([pagina.extract_text() for pagina in leitor_pdf.pages])
        texto_limpo = limpar_texto(texto)
        return texto_limpo
    except Exception as e:
        logger.error("Erro ao extrair texto do PDF: %s", e)
        return ""

# Função principal para processar o PDF e gerar o JSON do currículo
def main(caminho_pdf):
    texto_pdf = extrair_texto_de_pdf(caminho_pdf)
    if not texto_pdf:
        logger.info("Nenhum texto pôde ser extraído do PDF.")
        return None

    dados_curriculo = process_text(texto_pdf)
    if dados_curriculo:
        logger.info("Currículo processado com sucesso.")
        print(json.dumps(dados_curriculo, indent=4, ensure_ascii=False))
    else:
        logger.error("Falha ao processar o currículo.")



{'informacoes_pessoais': {'nome': 'Tiago Vettorazzi',
  'cidade': 'Fortaleza, Ceará, Brasil',
  'email': 'tiagovetto@gmail.com',
  'telefone': '',
  'cargo': 'Analista de Dados | BI | Big Data'},
 'resumo_qualificacoes': ['Python', 'Inglês', 'Microsoft Power BI'],
 'experiencia_profissional': [{'empresa': 'Grupo Portfolio',
   'cargo': 'Analista de BI',
   'periodo': 'julho de 2024 - Present',
   'local': 'Fortaleza',
   'atividades_exercidas': [],
   'ferramentas': ['Power BI']},
  {'empresa': 'Grupo Portfolio',
   'cargo': 'Estagiário de BI',
   'periodo': 'agosto de 2023 - julho de 2024',
   'local': 'Fortaleza',
   'atividades_exercidas': [],
   'ferramentas': ['Power BI']},
  {'empresa': 'SME/COGEP - Coordenadoria de Gestão de Pessoas da SME',
   'cargo': 'Assistente administrativo',
   'periodo': 'novembro de 2019 - outubro de 2021',
   'local': 'Fortaleza',
   'atividades_exercidas': [],
   'ferramentas': []}],
 'educacao': [{'instituicao': 'Universidade de Fortaleza - UNIFOR',


In [ ]:
conteudo

{'informacoes_pessoais': {'nome': 'Tiago Vettorazzi', 'cidade': 'Fortaleza, Ceará, Brasil', 'email': 'tiagovetto@gmail.com', 'telefone': '', 'cargo': 'Analista de Dados | BI | Big Data'}, 'resumo_qualificacoes': ['Python', 'Inglês', 'Microsoft Power BI', 'Formado em Administração de Empresas pela Universidade Estadual do Ceará (UECE)', 'Pós Graduado em Gestão Analítica com BI e Big Data'], 'experiencia_profissional': [{'empresa': 'Grupo Portfolio', 'cargo': 'Analista de BI', 'periodo': 'julho de 2024 - Present', 'local': 'Fortaleza', 'atividades_exercidas': [], 'ferramentas': []}, {'empresa': 'Grupo Portfolio', 'cargo': 'Estagiário de BI', 'periodo': 'agosto de 2023 - julho de 2024', 'local': 'Fortaleza', 'atividades_exercidas': [], 'ferramentas': []}, {'empresa': 'SME/COGEP - Coordenadoria de Gestão de Pessoas da SME', 'cargo': 'Assistente administrativo', 'periodo': 'novembro de 2019 - outubro de 2021', 'local': 'Fortaleza', 'atividades_exercidas': [], 'ferramentas': []}], 'educacao'

In [15]:
# dados
print(conteudo['informacoes_pessoais']['nome'])
# dados

Tiago Vettorazzi


In [8]:
CONTEUDO = {'Nome': 'Tiago Vettorazzi', 'formacao': [{'grau': 'MBA', 'curso': 'Gestão Analítica com BI e Big Data', 'instituicao': 'Universidade de Fortaleza - UNIFOR', 'conclusao': '2024'}, {'grau': 'Bacharel', 'curso': 'Administração de Empresas', 'instituicao': 'Universidade Estadual do Ceará', 'conclusao': '2022'}], 'competencias': ['Python', 'Inglês', 'Microsoft Power BI', 'SQL'], 'perfil_profissional': ['Tiago atua como Analista de BI no Grupo Portfolio em Fortaleza desde julho de 2024. Anteriormente, foi Estagiário de BI na mesma empresa de agosto de 2023 a julho de 2024. Antes disso, trabalhou como Assistente administrativo na SME/COGEP em Fortaleza de novembro de 2019 a outubro de 2021.', 'Possui competências em Python, Microsoft Power BI e SQL, além de proficiência em inglês. Participou de projetos de análise de dados e desenvolvimento de soluções de BI, aplicando técnicas avançadas de análise para otimizar processos e gerar insights estratégicos.'], 'disponibilidade': 'Hoje', 'modalidade': 'Remoto', 'dados_pessoais': '26 anos, solteiro, residente em Fortaleza', 'perfil_comportamental': '', 'responsavel': 'Tiago'}

In [9]:
CONTEUDO

{'Nome': 'Tiago Vettorazzi',
 'formacao': [{'grau': 'MBA',
   'curso': 'Gestão Analítica com BI e Big Data',
   'instituicao': 'Universidade de Fortaleza - UNIFOR',
   'conclusao': '2024'},
  {'grau': 'Bacharel',
   'curso': 'Administração de Empresas',
   'instituicao': 'Universidade Estadual do Ceará',
   'conclusao': '2022'}],
 'competencias': ['Python', 'Inglês', 'Microsoft Power BI', 'SQL'],
 'perfil_profissional': ['Tiago atua como Analista de BI no Grupo Portfolio em Fortaleza desde julho de 2024. Anteriormente, foi Estagiário de BI na mesma empresa de agosto de 2023 a julho de 2024. Antes disso, trabalhou como Assistente administrativo na SME/COGEP em Fortaleza de novembro de 2019 a outubro de 2021.',
  'Possui competências em Python, Microsoft Power BI e SQL, além de proficiência em inglês. Participou de projetos de análise de dados e desenvolvimento de soluções de BI, aplicando técnicas avançadas de análise para otimizar processos e gerar insights estratégicos.'],
 'disponibi

In [10]:
CONTEUDO['responsavel']

'Tiago'